### Find albums in Spotify and add songs to a playlist

In [1]:
import requests
import json
!pip install pandas
import pandas as pd
!pip install jsontable
import jsontable

     |████████████████████████████████| 9.9 MB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 15.3 MB 88.2 MB/s eta 0:00:01


### Token Management
Right now I have to manually refresh the token via https://developer.spotify.com/console

In [4]:
TOKEN = "BQBdX2ZeuI51c3vLTVpTk-hnj8SDu-F-hJ6tAV6I-F2Zii_jBKR3A7ZVdri3yvQqqvuhZjOkssHQqpA_9aOOnNfJ4t1mryEcR5TLGBYF9_VOKuwlNveMqdCv_CNQo2d8nZVI3gJpJnxKxEr_A9r2LwClX14s5jNbLfU"

### Header Construction

In [26]:
acceptKey = "Accept"
acceptVal = "application/json"
contentTypeKey = "Content-Type"
contentTypeVal = "application/json"
authKey = "Authorization"
authVal = "Bearer {}".format(TOKEN)
headerList = [(acceptKey, acceptVal), (contentTypeKey, contentTypeVal), (authKey, authVal)]
headerDict = dict(headerList)

### Request Construction

In [27]:
queryURL = "search?q=album:%22bad%20blood%22%20artist:bastille&type=album"
target = "https://api.spotify.com/v1/{}".format(queryURL)
request = requests.get(target, headers=althead).json()
print(json.dumps(request, indent=2))

{
  "albums": {
    "href": "https://api.spotify.com/v1/search?query=album%3A%22bad+blood%22+artist%3Abastille&type=album&offset=0&limit=20",
    "items": [
      {
        "album_type": "album",
        "artists": [
          {
            "external_urls": {
              "spotify": "https://open.spotify.com/artist/7EQ0qTo7fWT7DPxmxtSYEc"
            },
            "href": "https://api.spotify.com/v1/artists/7EQ0qTo7fWT7DPxmxtSYEc",
            "id": "7EQ0qTo7fWT7DPxmxtSYEc",
            "name": "Bastille",
            "type": "artist",
            "uri": "spotify:artist:7EQ0qTo7fWT7DPxmxtSYEc"
          }
        ],
        "available_markets": [
          "US"
        ],
        "external_urls": {
          "spotify": "https://open.spotify.com/album/64fQ94AVziavTPdnkCS6Nj"
        },
        "href": "https://api.spotify.com/v1/albums/64fQ94AVziavTPdnkCS6Nj",
        "id": "64fQ94AVziavTPdnkCS6Nj",
        "images": [
          {
            "height": 640,
            "url": "https:/

### Pull Album ID Out of Response
Parse some key fields, and sort by both release date and number of tracks

In [28]:
paths = [{"$.albums.items.id":"ID"}, {"$.albums.items.name":"Album Name"}, {"$.albums.items.total_tracks":"Tracks"}, {"$.albums.items.release_date":"Release Date"}]
converter = jsontable.converter()
converter.set_paths(paths)
resultTable = converter.convert_json(request)
df = pd.DataFrame(data=resultTable)
df.columns = ["Album ID", "Album Name", "Tracks", "Release Date"]
df = df.drop(df.index[0])
df = df.sort_values(["Release Date", "Tracks"], ascending = (False, False))
newest = pd.DataFrame(data=df.iloc[[0]])
newAlbums = pd.DataFrame()
newAlbums = newAlbums.append(newest)
newAlbums.style